# Tour mode choice 

This notebooks runs 20.000 households (random sample) to calibrate the tour mode choice. It first runs all the models up to "tour_mode_choice_simulate". At this point, we will run a grid search-like algorithm to re-calibrate the coefficients for the tour mode choice
    

In [28]:
import sys
import logging
import pandas as pd
import numpy as np

from activitysim.core import mem
from activitysim.core import inject
from activitysim.core import tracing
from activitysim.core import config
from activitysim.core import pipeline
from activitysim.core import mp_tasks
from activitysim.core import chunk

## Setting configurations files 

In [3]:
#Define injectables
inject.add_injectable('data_dir', 'data')
inject.add_injectable('configs_dir', ['configs', 'configs/configs'])
inject.add_injectable('output_dir', 'partial_tables')
# inject.get_injectable('configs_dir')

#Injectables list 
injectables = ['configs_dir', 'data_dir', 'output_dir']

#Configs files
run_list = mp_tasks.get_run_list()

# Injectables to dict 
injectables = list(set(injectables) | set(['data_dir', 'configs_dir', 'output_dir']))
injectables = {k: inject.get_injectable(k) for k in injectables}

## Running model for the first time 

In [4]:
%%time 
# Running models for the first time (current Mode choice specification)
pipeline.run(models = run_list['models'])
pipeline.close_pipeline()

numpy: Warning: underflow encountered in exp expression: _decay = _rt_available * exp(_trPkTime * dispersion_parameter_transit)
numpy: Warning: underflow encountered in exp expression: _decay = _rt_available * exp(_trOpTime * dispersion_parameter_transit)
numpy: Warning: underflow encountered in exp expression: _decay = _rt_available * exp(_nmDist * dispersion_parameter_walk)


CPU times: user 7min 52s, sys: 48.1 s, total: 8min 40s
Wall time: 7min 14s


## Grid search-like algortighm 

In [14]:
%%time 
# Running the just the mode choice models (Configs files would need to be changed)
pipeline.run(models = run_list['models'], resume_after = 'non_mandatory_tour_scheduling')
pipeline.close_pipeline()

CPU times: user 16.8 s, sys: 1.26 s, total: 18 s
Wall time: 14.8 s


## Big picture: 
- Create a function that calculates the best coefficients 
    grid_search_iterations()
    - Iteration 1: 
        run_time: 
        best_score: 
        score_reduction: 

- Select the variables that I may want to change 
- Create random vectors with those variable (or random search) 
- Try each random vector 
- Define the target distributions (this is very simplistic because it is just the mode choice share)
- Reduce range for random vector generations (average of the best 10/20th scores) #I need to make this decison 
- Generate new random vectors 
- Try each vector and get restuls 

Diferences from the car_ownership model: 
- This time I take advantage of the modeling framework to run the simulations instead of coding the simulations myself. This is desirable because the mode choice has many variables to look at, and there is not a sinlge table that summarize the variables the model uses. 

In [74]:
coeffs = pd.read_csv('configs/configs/tour_mode_choice_coeffs.csv', 
                     index_col = 'Expression')

In [107]:
def transpose(df):
    ''' Drops the "atwork subtour colum, and transporse the resutling dataframe"'''
    return df.drop(columns= 'atwork').T

In [108]:
def generate_random_sample(coeffs, var_names):
    '''
    Returns a random sample of coefficients (only those in var_names) within 30% of the original value. 
    For example. If original coefficient is 1, it will create a set from (0,7 to 1.3), and 
    will get a random number within this range. 
    
    coeffs: pandas DataFrame 
    var_names: list. list of the variables names that want to be modified.
               all names in var_names must be a header of coeffs. 
    '''
    coeffs_t = coeffs.pipe(transpose)
    shape_vars = coeffs_t[var_names].shape
    values = coeffs_t[var_names].unstack().droplevel(level= 0)
    variables = [np.linspace(float(x)* 0.7, float(x) * 1.3, 11) for x in values]
    random_set = [np.random.choice(var, 1, replace = True) for var in variables]
    random_set = np.array(random_set).reshape(shape_vars, order = 'F')
    return random_set

In [73]:
a = coeffs.pipe(transpose)

In [53]:
# Function to create 

In [17]:
walk_asc = ['walk_ASC_no_auto', 'walk_ASC_auto_decicient', 'walk_ASC_auto_sufficient',
            'joint_walk_ASC_no_auto', 'joint_walk_ASC_auto_decicient', 
            'joint_walk_ASC_auto_sufficient', 'sr2_ASC_auto_decicient',
            'sr2_ASC_auto_sufficient', 'sr3p_ASC_no_auto', 
            'sr3p_ASC_auto_decicient','sr3p_ASC_auto_sufficient']
a[walk_asc]

Expression,walk_ASC_no_auto,walk_ASC_auto_decicient,walk_ASC_auto_sufficient,joint_walk_ASC_no_auto,joint_walk_ASC_auto_decicient,joint_walk_ASC_auto_sufficient,sr2_ASC_auto_decicient,sr2_ASC_auto_sufficient,sr3p_ASC_no_auto,sr3p_ASC_auto_decicient,sr3p_ASC_auto_sufficient
eatout,5.125117111,3.274605133,1.551690381,-0.21274701,-1.960770582,-3.235215607,0.588234462,0.862805535,0.321999775,0.046052359,0.846859553
escort,2.801206737,-0.902046556,-0.811606606,-0.21274701,-1.960770582,-3.235215607,0,0,-1.81292667,-0.408187662,-0.057412532
othdiscr,3.266594735,2.249407533,1.263347649,-0.21274701,-1.960770582,-3.235215607,0.660151286,0.496846169,0.272169027,1.047096588,0.588502059
othmaint,1.28729901,1.369040422,0.799963401,-0.21274701,-1.960770582,-3.235215607,0.262152713,0.25817882,-0.803185378,-1.349392509,-0.075498672
school,18.41455742,3.257362412,0.647685583,-0.21274701,-1.960770582,-3.235215607,0.124743645,-1.606265705,-6.024082818,0.714957094,-1.020193438
shopping,2.376877423,2.270173198,0.731266314,-0.21274701,-1.960770582,-3.235215607,0.244097554,0.197707068,-0.279789475,-0.073370165,-0.077571296
social,1.868091482,2.870183935,1.707218614,-0.21274701,-1.960770582,-3.235215607,1.855852789,0.523602468,-1.403690228,1.50072437,0.506178835
univ,6.408966934,4.505909886,1.060766418,-0.21274701,-1.960770582,-3.235215607,-1.692234683,-1.859427005,-6.05600139,-1.727742177,-1.904709832
work,5.767215561,2.401041643,0.053265336,-0.21274701,-1.960770582,-3.235215607,-0.338031245,-1.085745845,-0.583126911,-0.852704253,-1.469970203


In [55]:
# Function to create vectors for the variables 
values = a[['walk_ASC_no_auto', 'walk_ASC_auto_decicient']].unstack().droplevel(level= 0)

In [62]:
shape_vars = a[['walk_ASC_no_auto', 'walk_ASC_auto_decicient']].shape

In [26]:
a['walk_ASC_no_auto'].astype(float) * 1.3

eatout       6.662652
escort       3.641569
othdiscr     4.246573
othmaint     1.673489
school      23.938925
shopping     3.089941
social       2.428519
univ         8.331657
work         7.497380
Name: walk_ASC_no_auto, dtype: float64

In [56]:
variables = [np.linspace(float(a)*0.7, float(a)*1.3, 11) for a in values]

In [64]:
random_set = [np.random.choice(var, 1, replace = True) for var in variables]
random_set = np.array(random_set).reshape(shape_vars)

In [65]:
random_set

array([[ 3.895089  ,  3.30542395],
       [ 2.67860768,  1.44177489],
       [17.30968397,  2.51949007],
       [ 1.6439205 ,  6.40896693],
       [ 5.76721556,  3.07812883],
       [-0.90204656,  2.92422979],
       [ 1.53332527,  2.28015369],
       [ 1.58912124,  2.00912875],
       [ 4.77626448,  2.40104164]])

In [100]:
def generate_random_sample(coeffs, var_names):
    coeffs_t = coeffs.pipe(transpose)
    shape_vars = coeffs_t[var_names].shape
    values = coeffs_t[var_names].unstack().droplevel(level= 0)
    variables = [np.linspace(float(x)* 0.7, float(x) * 1.3, 11) for x in values]
    random_set = [np.random.choice(var, 1, replace = True) for var in variables]
    random_set = np.array(random_set).reshape(shape_vars, order = 'F')
    return random_set

In [101]:
walk_asc = ['walk_ASC_no_auto', 'walk_ASC_auto_decicient', 'walk_ASC_auto_sufficient',]
#             'joint_walk_ASC_no_auto', 'joint_walk_ASC_auto_decicient', 
#             'joint_walk_ASC_auto_sufficient', 'sr2_ASC_auto_decicient',
#             'sr2_ASC_auto_sufficient', 'sr3p_ASC_no_auto', 
#             'sr3p_ASC_auto_decicient','sr3p_ASC_auto_sufficient']
random_values = generate_random_sample(coeffs, walk_asc)
random_values

array([[ 6.04763819,  2.88165252,  1.17928469],
       [ 2.12891712, -0.73967818, -0.860303  ],
       [ 2.28661631,  2.51933644,  1.41494937],
       [ 1.51901283,  1.69761012,  0.70396779],
       [12.89019019,  2.47559543,  0.64768558],
       [ 1.6638142 ,  1.99775241,  0.81901827],
       [ 2.09226246,  2.6979729 ,  1.70721861],
       [ 5.25535289,  5.85768285,  0.74253649],
       [ 4.72911676,  2.54510414,  0.05646126]])

In [102]:
b = a[walk_asc]

In [103]:
a[walk_asc] = random_values

In [104]:
a[walk_asc]

Expression,walk_ASC_no_auto,walk_ASC_auto_decicient,walk_ASC_auto_sufficient
eatout,6.04764,2.88165,1.17928
escort,2.12892,-0.739678,-0.860303
othdiscr,2.28662,2.51934,1.41495
othmaint,1.51901,1.69761,0.703968
school,12.8902,2.4756,0.647686
shopping,1.66381,1.99775,0.819018
social,2.09226,2.69797,1.70722
univ,5.25535,5.85768,0.742536
work,4.72912,2.5451,0.0564613


In [32]:
hdf = pd.HDFStore('partial_tables/pipeline.h5')

In [31]:
hdf.close()

In [33]:
hdf.keys()

['/checkpoints',
 '/workplace_modeled_size/workplace_location',
 '/workplace_destination_size/initialize_households',
 '/tours/joint_tour_composition',
 '/tours/joint_tour_destination',
 '/tours/joint_tour_frequency',
 '/tours/joint_tour_participation',
 '/tours/joint_tour_scheduling',
 '/tours/mandatory_tour_frequency',
 '/tours/mandatory_tour_scheduling',
 '/tours/non_mandatory_tour_destination',
 '/tours/non_mandatory_tour_frequency',
 '/tours/non_mandatory_tour_scheduling',
 '/tours/tour_mode_choice_simulate',
 '/school_modeled_size/school_location',
 '/school_destination_size/initialize_households',
 '/persons/cdap_simulate',
 '/persons/free_parking',
 '/persons/initialize_households',
 '/persons/joint_tour_participation',
 '/persons/mandatory_tour_frequency',
 '/persons/non_mandatory_tour_frequency',
 '/persons/school_location',
 '/persons/workplace_location',
 '/person_windows/initialize_households',
 '/person_windows/joint_tour_scheduling',
 '/person_windows/mandatory_tour_sche

In [34]:
hdf['/checkpoints']

,checkpoint_name,land_use,accessibility,timestamp,person_windows,households,persons,school_destination_size,workplace_destination_size,school_modeled_size,workplace_modeled_size,tours,joint_tour_participants
0,initialize_landuse,initialize_landuse,initialize_landuse,2020-09-22 22:57:31.874133,,,,,,,,,
1,compute_accessibility,initialize_landuse,compute_accessibility,2020-09-22 22:57:33.075405,,,,,,,,,
2,initialize_households,initialize_landuse,compute_accessibility,2020-09-22 22:57:36.801316,initialize_households,initialize_households,initialize_households,initialize_households,initialize_households,,,,
3,school_location,initialize_landuse,compute_accessibility,2020-09-22 22:58:03.635989,initialize_households,initialize_households,school_location,initialize_households,initialize_households,school_location,,,
4,workplace_location,initialize_landuse,compute_accessibility,2020-09-22 22:58:57.801867,initialize_households,workplace_location,workplace_location,initialize_households,initialize_households,school_location,workplace_location,,
5,auto_ownership_simulate,initialize_landuse,compute_accessibility,2020-09-22 22:58:58.306208,initialize_households,auto_ownership_simulate,workplace_location,initialize_households,initialize_households,school_location,workplace_location,,
6,free_parking,initialize_landuse,compute_accessibility,2020-09-22 22:58:58.849842,initialize_households,auto_ownership_simulate,free_parking,initialize_households,initialize_households,school_location,workplace_location,,
7,cdap_simulate,initialize_landuse,compute_accessibility,2020-09-22 22:59:11.684322,initialize_households,cdap_simulate,cdap_simulate,initialize_households,initialize_households,school_location,workplace_location,,
8,mandatory_tour_frequency,initialize_landuse,compute_accessibility,2020-09-22 22:59:13.222150,initialize_households,cdap_simulate,mandatory_tour_frequency,initialize_households,initialize_households,school_location,workplace_location,mandatory_tour_frequency,
9,mandatory_tour_scheduling,initialize_landuse,compute_accessibility,2020-09-22 23:01:39.162504,mandatory_tour_scheduling,cdap_simulate,mandatory_tour_frequency,initialize_households,initialize_households,school_location,workplace_location,mandatory_tour_scheduling,


In [35]:
hdf['/tours/tour_mode_choice_simulate']

,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,start,end,duration,tdd,composition,tour_mode
tour_id,,,,,,,,,,,,,,,,,
4951,120,school,1,1,1,1,mandatory,1,71.0,191,72,18.0,21.0,3.0,172.0,,DRIVEALONEFREE
5443,132,school,1,1,1,1,mandatory,1,71.0,191,84,17.0,20.0,3.0,165.0,,DRIVEALONEFREE
9059,220,work,1,1,1,1,mandatory,1,191.0,191,144,8.0,17.0,9.0,63.0,,WALK
14635,356,work,1,1,1,1,mandatory,1,950.0,191,280,7.0,19.0,12.0,49.0,,DRIVEALONEPAY
16439,400,work,1,1,1,1,mandatory,1,191.0,107,314,10.0,21.0,11.0,96.0,,WALK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73085749,1782579,escort,2,2,2,2,non_mandatory,1,48.0,287,739550,17.0,17.0,0.0,162.0,,SHARED3PAY
73100710,1782944,eatout,1,1,1,1,non_mandatory,1,579.0,38,739694,12.0,15.0,3.0,115.0,,SHARED3PAY
73133901,1783753,othmaint,1,1,1,1,non_mandatory,1,57.0,239,740011,17.0,17.0,0.0,162.0,,SHARED2FREE
